## Import Dependancies

In [2]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\subit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
# stopwords: the words which doesn't hold that much meaning
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data collection and Data processing

In [5]:

#loading the dataset to pandas dataframe
news_dataset = pd.read_csv('train.csv')

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_dataset.shape

(20800, 5)

### check for missing values

In [9]:

news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

### handling the missing value
replace missing value by empty string

In [10]:

news_dataset = news_dataset.fillna('')


In [12]:
# merging the title and author column
news_dataset['content'] = news_dataset['author'] +' '+news_dataset['title']

In [14]:
news_dataset['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [15]:
# Seperating the data and the target
X = news_dataset.drop('label',axis=1)
Y= news_dataset['label']

In [16]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

### Stemming
It is a process of reducing a word to it's root word


In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # remove all the non-alphabetic items i.e symbols or numbers
    stemmed_content = stemmed_content.lower() # converting all the letters to the lower case
    stemmed_Content = stemmed_content.split() # converting the content to list
    # remove all the stopwords and reduce each word to its root word
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ''.join(stemmed_content) # combine to sentence again
    return stemmed_content

In [22]:
news_dataset['content'] =news_dataset['content'].apply(stemming)

In [24]:
# seperating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [25]:
print(X)
Y

['rrell lucu hue e e  we n  even ee ce  leer unl jn chffez weee '
 'nel j  flnn flnn  hllr clnn  bg wn n cpu   brebr'
 'cnrunew c wh he ruh gh ge u fre' ...
 'chel j  e l erce n rchel br c     receve kever pprch b hun  b   he new rk e'
 'lex nr n  ru  hl prllel exerce n blkn' 'v wnn wh keep he f    lve']


array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

### Convert the text to numerical values 
function used is TfidfVectorizer:</br></br>
tf(term frequency): Counts how many times a word is repeating in a sentence, that means that word is important and it add the numerical values according to that,</br></br>
idf(inverse document frequency): Find the word which is repeating more and it doesn't hold that imporatance and reduce its numerical value


In [27]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X) # convert the text to value i.e. feature vector

In [29]:
print(X)

  (0, 10224)	0.4096436007623204
  (0, 10210)	0.20258932711637465
  (0, 9571)	0.31314059780522296
  (0, 8967)	0.3625509115748654
  (0, 6077)	0.3662807075862616
  (0, 5718)	0.23614813385383396
  (0, 5163)	0.22372297445904393
  (0, 4931)	0.22153826661276715
  (0, 3067)	0.2369034945974848
  (0, 2280)	0.21321565003888202
  (0, 1122)	0.3703662368673576
  (0, 927)	0.183944252650126
  (1, 10487)	0.20300930288188168
  (1, 6341)	0.2447189786615966
  (1, 4685)	0.19766820957295358
  (1, 3529)	0.7414960883915208
  (1, 1745)	0.3972874374370028
  (1, 1425)	0.19820745669034365
  (1, 512)	0.16112278934163615
  (1, 249)	0.294833202289685
  (2, 10337)	0.2276209318167095
  (2, 9051)	0.43250653988646354
  (2, 4403)	0.12147695101958758
  (2, 3989)	0.48464493076180765
  (2, 3874)	0.3239624264393102
  :	:
  (20796, 1111)	0.4524399970399514
  (20796, 173)	0.376060055303263
  (20797, 8769)	0.11349022421118699
  (20797, 8188)	0.38365041419227686
  (20797, 8105)	0.337649863330323
  (20797, 7516)	0.432886913516616

## Splitting dataset into training and test sets

In [30]:

X_train, X_test, Y_train, Y_test =train_test_split(X,Y,test_size=0.2, stratify = Y, random_state=2)

In [32]:
print(X.shape,X_train.shape,X_test.shape)
print(Y.shape,Y_train.shape,Y_test.shape)

(20800, 10777) (16640, 10777) (4160, 10777)
(20800,) (16640,) (4160,)


## training the model
Used <b>Logistic Regression</b> for building the model</br>
Sigmoid function:</br>
    Y = 1/1+e**-Z,</br>
    Z= w*X +b,</br>
    X: Input Features,</br> 
    Y: Prediction Probability,</br>
    w: weights(depends upon the importance of each column or feature),</br>
    b: bias


In [33]:

model = LogisticRegression()



In [34]:
model.fit(X_train,Y_train)

LogisticRegression()

## Model Evaluation

In [35]:
#Accurarcy score for trained data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [36]:
print("Accuracy Score of trained data: ", training_data_accuracy)

Accuracy Score of trained data:  0.9753605769230769


In [37]:
#Accurarcy score for test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [38]:
print("Accuracy Score of test data: ", test_data_accuracy)

Accuracy Score of test data:  0.9634615384615385


## Build a predictive system

In [44]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)
if prediction[0] == 0:
    print("News is real")
else:
    print("New is fake")
print(Y_test[0]) # actual label

[1]
New is fake
1
